## MODELO DE REGRESIÓN LOGÍSTICA PARA CLASIFICACIÓN DE SOLICITUDES DE CRÉDITO
#### NOTEBOOK: Entrenar_Clasificador

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, when, count, isnan, isnull, expr
from pyspark.sql.types import *
import requests
import os
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Crear o obtener la sesión de Spark
spark = SparkSession.builder \
    .appName("ClasificadorCredito") \
    .getOrCreate()

print("Sesión de Spark activa:", spark)

# Cargar datos con manejo de errores
try:
    df = spark.read.format("delta").load("Tables/solicitudes_processed")
    print(f"Datos cargados exitosamente. Total de registros: {df.count()}")
except Exception as e:
    raise Exception(f"Error al cargar los datos: {str(e)}")

# Filtrar solo Aprobado/Rechazado y crear etiquetas
df = df.filter(col("estado_solicitud").isin("Aprobado", "Rechazado"))
df = df.withColumn("label", when(col("estado_solicitud") == "Aprobado", 1.0).otherwise(0.0))

# Verificar distribución de clases
class_distribution = df.groupBy("estado_solicitud").count()
print("Distribución de clases:")
class_distribution.show()

# Definir columnas de entrada
input_cols = ["edad", "ingresos_anuales", "puntaje_crediticio", "deuda_actual",
              "antiguedad_laboral", "historial_pagos_encoded", "estado_civil_encoded",
              "tipo_empleo_encoded", "numero_dependientes"]

# Análisis de valores nulos antes de la limpieza
print("\nAnálisis de valores nulos antes de la limpieza:")
for column in input_cols:
    null_count = df.filter(col(column).isNull() | isnan(col(column))).count()
    total = df.count()
    null_percentage = (null_count / total) * 100
    print(f"{column}: {null_count} nulos ({null_percentage:.2f}%)")

# Manejar valores nulos con estrategia más sofisticada
for col_name in input_cols:
    # Usar la mediana para variables numéricas
    if col_name in ["edad", "ingresos_anuales", "puntaje_crediticio", "deuda_actual", "antiguedad_laboral"]:
        median_value = df.approxQuantile(col_name, [0.5], 0.01)[0]
        df = df.fillna(median_value, subset=[col_name])
    else:
        # Usar 0 para variables categóricas codificadas
        df = df.fillna(0, subset=[col_name])

# Verificar que no queden nulos
null_check = df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in input_cols])
print("\nVerificación final de valores nulos:")
null_check.show()

# Dividir datos con estratificación
train_df, test_df = df.randomSplit([0.8, 0.2], seed=123)
print(f"Datos de entrenamiento: {train_df.count()} registros")
print(f"Datos de prueba: {test_df.count()} registros")

# Preparar características
assembler = VectorAssembler(
    inputCols=input_cols,
    outputCol="features",
    handleInvalid="skip"  # Manejo explícito de valores inválidos
)

# Configurar modelo con hiperparámetros y umbral personalizado
lr = LogisticRegression(
    labelCol="label",
    featuresCol="features",
    maxIter=10,
    regParam=0.1,  # Regularización L2
    elasticNetParam=0.2,  # Elastic Net mixing (0=Ridge, 1=Lasso)
    standardization=True,  # Estandarizar características
    threshold=0.5  # Umbral de clasificación inicial
)

# Crear y entrenar pipeline
pipeline = Pipeline(stages=[assembler, lr])
try:
    print("Iniciando entrenamiento del modelo...")
    clf_model = pipeline.fit(train_df)
    print("Modelo entrenado exitosamente")
except Exception as e:
    raise Exception(f"Error durante el entrenamiento: {str(e)}")

# Evaluación exhaustiva del modelo
predictions = clf_model.transform(test_df)

# Múltiples métricas de evaluación
metrics = {}

# ROC AUC
evaluator_auc = BinaryClassificationEvaluator(
    labelCol="label",
    metricName="areaUnderROC"
)
metrics['auc'] = evaluator_auc.evaluate(predictions)

# Precisión
evaluator_precision = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="weightedPrecision"  # Cambiado de "precision" a "weightedPrecision"
)
metrics['precision'] = evaluator_precision.evaluate(predictions)

# Recall
evaluator_recall = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="weightedRecall"  # Cambiado de "recall" a "weightedRecall"
)
metrics['recall'] = evaluator_recall.evaluate(predictions)

# Calcular y visualizar matriz de confusión
def calcular_matriz_confusion(predictions):
    # Convertir predicciones a pandas para mejor manipulación
    pred_pd = predictions.select(['label', 'prediction']).toPandas()
    conf_matrix = pd.crosstab(pred_pd['label'], pred_pd['prediction'], margins=True)
    
    # Calcular métricas específicas
    tn = conf_matrix.iloc[0,0]
    fp = conf_matrix.iloc[0,1]
    fn = conf_matrix.iloc[1,0]
    tp = conf_matrix.iloc[1,1]
    
    # Calcular métricas adicionales
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    f1_score = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
    
    return conf_matrix, {'specificity': specificity, 'npv': npv, 'f1_score': f1_score}

# Función para encontrar el mejor umbral
def optimizar_umbral(predictions, num_thresholds=10):
    thresholds = np.linspace(0.1, 0.9, num_thresholds)
    resultados = []
    
    pred_pd = predictions.select(['label', 'probability']).toPandas()
    y_true = pred_pd['label']
    probas = np.array([p[1] for p in pred_pd['probability']])
    
    for threshold in thresholds:
        y_pred = (probas >= threshold).astype(int)
        tn = sum((y_true == 0) & (y_pred == 0))
        fp = sum((y_true == 0) & (y_pred == 1))
        fn = sum((y_true == 1) & (y_pred == 0))
        tp = sum((y_true == 1) & (y_pred == 1))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        resultados.append({
            'threshold': threshold,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })
    
    return pd.DataFrame(resultados)

# Calcular matriz de confusión y métricas adicionales
conf_matrix, metricas_adicionales = calcular_matriz_confusion(predictions)
print("\nMatriz de Confusión:")
print(conf_matrix)
print("\nMétricas Adicionales:")
for nombre, valor in metricas_adicionales.items():
    print(f"{nombre}: {valor:.4f}")

# Optimizar umbral
resultados_umbrales = optimizar_umbral(predictions)
mejor_umbral = resultados_umbrales.loc[resultados_umbrales['f1_score'].idxmax()]
print(f"\nMejor umbral encontrado: {mejor_umbral['threshold']:.3f} (F1-Score: {mejor_umbral['f1_score']:.3f})")

# Actualizar métricas para incluir la matriz de confusión
metrics['specificity'] = metricas_adicionales['specificity']
metrics['npv'] = metricas_adicionales['npv']
metrics['f1_score'] = metricas_adicionales['f1_score']
metrics['mejor_umbral'] = float(mejor_umbral['threshold'])

# Imprimir métricas
print("\nMétricas de evaluación:")
for metric_name, value in metrics.items():
    print(f"{metric_name.upper()}: {value:.4f}")

# Guardar modelo con manejo de errores
try:
    # Intentar guardar en una ruta del lakehouse
    model_path = "Tables/Models/ClasificadorCredito"
    print(f"Intentando guardar el modelo en: {model_path}")
    clf_model.write().overwrite().save(model_path)
    print(f"Modelo guardado exitosamente en {model_path}")
except Exception as e:
    print(f"Error al guardar el modelo en el lakehouse: {str(e)}")
    # Intentar guardar en una ruta relativa
    try:
        local_path = "./models/ClasificadorCredito"
        print(f"Intentando guardar el modelo en ruta relativa: {local_path}")
        clf_model.write().overwrite().save(local_path)
        print(f"Modelo guardado exitosamente en ruta relativa: {local_path}")
    except Exception as e2:
        print(f"Error al guardar en ruta relativa: {str(e2)}")
        print("Sugerencia: Verifica los permisos y la configuración del lakehouse en Fabric")
        raise

# Actualizar el DataFrame de métricas para incluir nuevas métricas
metrics_df = spark.createDataFrame([(
    datetime.now(),
    float(metrics['auc']),
    float(metrics['precision']),
    float(metrics['recall']),
    float(metrics['specificity']),
    float(metrics['npv']),
    float(metrics['f1_score']),
    float(metrics['mejor_umbral'])
)], ["fecha_entrenamiento", "auc", "precision", "recall", "specificity", "npv", "f1_score", "mejor_umbral"])

# Guardar métricas actualizadas
metrics_df.write.mode("append").format("delta").save("Tables/precision_clasificador")

# Notificar resultados (usando variable de entorno para el webhook)
discord_webhook_url = os.getenv('DISCORD_WEBHOOK_URL', "XXXXXXXXXXXXXXXXXXXX")

# Crear mensaje de éxito
def crear_mensaje_exito(metrics):
    return {
        "embeds": [
            {
                "title": "✅ Reentrenamiento Exitoso - RiskApp",
                "description": "El modelo de clasificación ha sido reentrenado exitosamente.",
                "color": 3066993,
                "fields": [
                    {"name": "AUC", "value": f"{metrics['auc']:.4f}", "inline": True},
                    {"name": "Precisión", "value": f"{metrics['precision']:.4f}", "inline": True},
                    {"name": "Recall", "value": f"{metrics['recall']:.4f}", "inline": True},
                    {"name": "Especificidad", "value": f"{metrics['specificity']:.4f}", "inline": True},
                    {"name": "VPN", "value": f"{metrics['npv']:.4f}", "inline": True},
                    {"name": "F1-Score", "value": f"{metrics['f1_score']:.4f}", "inline": True},
                    {"name": "Mejor Umbral", "value": f"{metrics['mejor_umbral']:.3f}", "inline": True}
                ],
                "footer": {"text": "RiskApp - Sistema de Monitoreo"},
                "timestamp": datetime.now().isoformat()
            }
        ]
    }

# Crear mensaje de error
def crear_mensaje_error(error):
    return {
        "embeds": [
            {
                "title": "❌ Error en el Reentrenamiento - RiskApp",
                "description": f"Se produjo un error durante el reentrenamiento: {error}",
                "color": 15158332,  # Rojo
                "footer": {"text": "RiskApp - Sistema de Monitoreo"},
                "timestamp": datetime.now().isoformat()
            }
        ]
    }

# Enviar notificación
def enviar_notificacion(mensaje):
    try:
        response = requests.post(discord_webhook_url, json=mensaje)
        response.raise_for_status()
        print("\nNotificación enviada exitosamente a Discord")
    except Exception as e:
        print(f"Error al enviar notificación: {str(e)}")

# Enviar mensaje de éxito o error
try:
    mensaje = crear_mensaje_exito(metrics)
    enviar_notificacion(mensaje)
except Exception as e:
    mensaje = crear_mensaje_error(str(e))
    enviar_notificacion(mensaje)